# LSTM-VAE pro

In [1]:
from model.bagging_lstmvae_pro import *
import torch.utils.data as data_utils
from utils.eval_methods import *
from sklearn import preprocessing

device = get_default_device()

min_max_scaler = preprocessing.MinMaxScaler()
# Read data
normal = pd.read_csv("data/SWaT/SWaT_Dataset_Normal_v1.csv", nrows=10000)  # , nrows=1000)
normal = normal.drop(["Timestamp", "Normal/Attack"], axis=1)
# Transform all columns into float64
for i in list(normal):
    normal[i] = normal[i].apply(lambda x: str(x).replace(",", "."))
normal = normal.astype(float)
# 数据归一化
x = normal.values
x_scaled = min_max_scaler.fit_transform(x)
normal = pd.DataFrame(x_scaled)

# Read data
attack = pd.read_csv("data/SWaT/SWaT_Dataset_Attack_v0.csv", sep=";", nrows=10000)  # , nrows=1000)
labels = [float(label != 'Normal') for label in attack["Normal/Attack"].values]
attack = attack.drop(["Timestamp", "Normal/Attack"], axis=1)
# Transform all columns into float64
for i in list(attack):
    attack[i] = attack[i].apply(lambda x: str(x).replace(",", "."))
attack = attack.astype(float)
x = attack.values
x_scaled = min_max_scaler.transform(x)
attack = pd.DataFrame(x_scaled)

In [2]:
############## windows ###################
window_size = 12
# np.arange(window_size)[None, :] 1*12 (0,1,2,3,4,5,6,7,8,9,10,11)一行12列
# np.arange(normal.shape[0] - window_size)[:, None] (1000-12)*1 (0,1,2,3,4,5...) 988列，每列递增
# np.arange(window_size)[None, :] + np.arange(normal.shape[0] - window_size)[:, None] (1000-12)*12
windows_normal = normal.values[np.arange(window_size)[None, :] + np.arange(attack.shape[0] - window_size)[:, None]]
windows_attack = attack.values[np.arange(window_size)[None, :] + np.arange(attack.shape[0] - window_size)[:, None]]

windows_labels=[]
for i in range(len(labels)-window_size):
    windows_labels.append(list(np.int_(labels[i:i+window_size])))
y_test = [1.0 if (np.sum(window) > 0) else 0 for window in windows_labels]
y_test = np.array(y_test)

In [3]:
############## training ###################
BATCH_SIZE = 500
N_EPOCHS = 10
N = 5 * round((normal.shape[1] / 3) / 5)  # 10 for both bootstrap sample size and number of estimators
decoder_layers = 2  # number of hidden layers for each decoder
z = int((N / 2) - 1)  # size of latent space

windows_normal_train = windows_normal[:int(np.floor(.8 * windows_normal.shape[0]))]
windows_normal_val = windows_normal[int(np.floor(.8 * windows_normal.shape[0])):int(np.floor(windows_normal.shape[0]))]

train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_normal_train).float().view(
        ([windows_normal_train.shape[0], windows_normal_train.shape[1], windows_normal_train.shape[2]]))
), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_normal_val).float().view(
        ([windows_normal_val.shape[0], windows_normal_train.shape[1], windows_normal_train.shape[2]]))
), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_attack).float().view(
        ([windows_attack.shape[0], windows_attack.shape[1], windows_attack.shape[2]]))
), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [4]:
model = BaggingLstmVAE(time_step=window_size,
                       input_dim=normal.shape[1],
                       hidden_size=N,
                       n_estimators=N,
                       max_features=N,
                       latent_dim=z,
                       decoding_depth=decoder_layers)
for i in range(model.n_estimators):
    model.LSTMVAEs[i] = to_device(model.LSTMVAEs[i], device)
    model.DivLstmVAEs[i] = to_device(model.DivLstmVAEs[i], device)

In [5]:
history = training(N_EPOCHS, model, train_loader)

VAE encoder开始训练
Epoch[0]  loss_vae: 19.3010
Epoch[1]  loss_vae: 4.3951
Epoch[2]  loss_vae: 0.6489
Epoch[3]  loss_vae: 0.2767
Epoch[4]  loss_vae: 0.1676
Epoch[5]  loss_vae: 0.1181
Epoch[6]  loss_vae: 0.0994
Epoch[7]  loss_vae: 0.0914
Epoch[8]  loss_vae: 0.0877
Epoch[9]  loss_vae: 0.0849
lower decoder, upper decoder开始训练
Epoch[0]  loss_low: 0.1924, loss_high: 0.2121
Epoch[1]  loss_low: 0.1839, loss_high: 0.2035
Epoch[2]  loss_low: 0.1742, loss_high: 0.1934
Epoch[3]  loss_low: 0.1625, loss_high: 0.1814
Epoch[4]  loss_low: 0.1490, loss_high: 0.1673
Epoch[5]  loss_low: 0.1338, loss_high: 0.1520
Epoch[6]  loss_low: 0.1174, loss_high: 0.1359
Epoch[7]  loss_low: 0.1008, loss_high: 0.1203
Epoch[8]  loss_low: 0.0849, loss_high: 0.1063
Epoch[9]  loss_low: 0.0701, loss_high: 0.0945


In [6]:
lower, upper = testing(model, test_loader)

In [7]:
# 点调整法
windows_attack = windows_attack[:, -1, :]
attack_tiles = np.tile(windows_attack.reshape(windows_attack.shape[0], 1, windows_attack.shape[1]), (1, N, 1))
result = np.where((attack_tiles < lower.numpy()) | (attack_tiles > upper.numpy()), 1, 0)
inference = np.mean(np.mean(result, axis=1), axis=1)
print(inference[0:100])
t, th = bf_search(inference, y_test, start=0, end=1, step_num=1000, display_freq=50)

[0.99738562 1.         0.99607843 0.99607843 0.99738562 0.99607843
 1.         0.99738562 0.99738562 0.99869281 0.99738562 0.99869281
 0.99607843 0.99346405 0.99738562 0.99346405 0.99738562 0.99084967
 0.99084967 0.99738562 0.99869281 0.99738562 0.99869281 0.99477124
 0.98954248 0.99477124 0.99738562 0.99869281 1.         1.
 0.99738562 0.99869281 0.99084967 0.99869281 0.99215686 0.99869281
 0.99607843 0.99607843 0.99607843 0.99869281 0.99738562 0.99607843
 0.99477124 0.99607843 1.         0.99738562 1.         0.99607843
 0.99869281 0.99738562 0.99869281 0.99869281 1.         0.99607843
 0.99869281 0.99477124 0.99869281 0.99869281 0.99215686 0.99869281
 0.99869281 0.99738562 0.99738562 0.99738562 0.99346405 0.99607843
 0.99477124 0.99084967 0.99738562 0.99215686 0.99477124 0.99869281
 0.99738562 0.99869281 0.99607843 1.         0.99607843 0.99215686
 0.99738562 0.99607843 0.99738562 0.99738562 0.99607843 0.99869281
 0.99738562 0.99869281 0.99869281 0.99869281 0.99607843 1.
 0.99607843

In [8]:
result = np.where((attack_tiles < upper.numpy()) | (attack_tiles > lower.numpy()), 1, 0)
inference = np.mean(np.mean(result, axis=1), axis=1)
print(inference[0:100])
t, th = bf_search(inference, y_test, start=0, end=1, step_num=1000, display_freq=50)

[0.90326797 0.90457516 0.9124183  0.89934641 0.90326797 0.89019608
 0.89281046 0.89019608 0.89150327 0.87581699 0.90457516 0.88627451
 0.88104575 0.90196078 0.89019608 0.88496732 0.88104575 0.90849673
 0.87973856 0.89019608 0.89281046 0.90849673 0.89411765 0.89411765
 0.88888889 0.90849673 0.89934641 0.89019608 0.89542484 0.89411765
 0.88366013 0.90065359 0.89281046 0.89803922 0.90980392 0.89150327
 0.89150327 0.88627451 0.89673203 0.88496732 0.89411765 0.8875817
 0.87712418 0.88888889 0.88496732 0.89542484 0.89411765 0.88888889
 0.90588235 0.90196078 0.90588235 0.89411765 0.8875817  0.89542484
 0.90196078 0.88366013 0.88627451 0.88235294 0.89803922 0.89803922
 0.90588235 0.89150327 0.88888889 0.90065359 0.88104575 0.8745098
 0.87320261 0.89019608 0.89150327 0.90457516 0.87973856 0.89542484
 0.8745098  0.8745098  0.8875817  0.88888889 0.88366013 0.88627451
 0.88888889 0.89411765 0.89803922 0.90588235 0.89673203 0.88496732
 0.87973856 0.89281046 0.89411765 0.88496732 0.88496732 0.896732

In [9]:
a = lower.numpy()
b = upper.numpy()
print(a)
print(b)

[[[0.9814215  0.8842538  0.97994334 ... 0.03057323 0.05085211 0.05523846]
  [0.9634274  0.88354987 0.9738182  ... 0.026672   0.06417917 0.04287743]
  [0.81425744 0.9056139  0.9370866  ... 0.09821445 0.03014469 0.1096047 ]
  ...
  [0.967357   0.9117299  0.974082   ... 0.03445207 0.07157819 0.05332107]
  [0.91060877 0.7434925  0.8914453  ... 0.09557724 0.08143618 0.18385789]
  [0.9292723  0.8870067  0.9612206  ... 0.03414477 0.07568916 0.02669688]]

 [[0.9107818  0.7585149  0.90382355 ... 0.09205294 0.16748507 0.15060695]
  [0.79299325 0.68165755 0.8299092  ... 0.21792054 0.29891777 0.15500174]
  [0.6914866  0.8025187  0.8658568  ... 0.19666906 0.14684328 0.14329772]
  ...
  [0.8279079  0.85081804 0.83305293 ... 0.12272205 0.1959774  0.10196887]
  [0.88998646 0.72423244 0.8527468  ... 0.13259816 0.14688675 0.22708566]
  [0.95148    0.84784865 0.97377306 ... 0.02876692 0.12866257 0.00825619]]

 [[0.9207756  0.7818596  0.90413266 ... 0.09629491 0.15099096 0.1526604 ]
  [0.7899895  0.668270

In [18]:
a = a.flatten()
b = b.flatten()

In [19]:
aUb, aLb, eq = [], [], []
for i in range(len(a)):
    if a[i] > b[i]:
        aUb.append(i)
    elif a[i] < b[i]:
        aLb.append(i)
    else:
        eq.append(i)

In [21]:
# print(aUb)
# print(aLb)
# print(eq)
print(len(aUb))
print(len(aLb))
print(len(eq))

5775159
1865661
0
